In [1]:
import os
import cv2
import numpy as np

# Step 1: Define the root directory
root_dir = "C:/Users/vashi/OneDrive/Desktop/video_dat/archive (2)/UCSD_Anomaly_Dataset.v1p2"

# Function to load preprocessed data
def load_preprocessed_data(dataset_dir, target_size=(256, 256)):
    images = []
    labels = []
    for filename in os.listdir(dataset_dir):
        if filename.endswith(".tif"):
            img_path = os.path.join(dataset_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, target_size)  # Resize image to target size
            label = int(filename.split('_')[2])
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Paths
dataset_dir = os.path.join(root_dir, "Dataset")

# Load data
images, labels = load_preprocessed_data(dataset_dir)

# Normalize images
images = images.astype('float32') / 255.0
images = np.expand_dims(images, axis=-1)

# Print shapes to verify
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")


Images shape: (18560, 256, 256, 1)
Labels shape: (18560,)


In [3]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

# Step 2: Split data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Step 3: Define STCAE model
def create_stcae_model(input_shape):
    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    return autoencoder

# Create the model
input_shape = (images.shape[1], images.shape[2], 1)
stcae_model = create_stcae_model(input_shape)

# Compile the model
stcae_model.compile(optimizer='adam', loss='binary_crossentropy')

# Step 4: Train the model
stcae_model.fit(train_images, train_images, epochs=5, batch_size=32, shuffle=True, validation_data=(val_images, val_images))


Epoch 1/5
464/464 ━━━━━━━━━━━━━━━━━━━━ 1350s 3s/step - loss: 0.6041 - val_loss: 0.5856
Epoch 2/5
464/464 ━━━━━━━━━━━━━━━━━━━━ 1361s 3s/step - loss: 0.5861 - val_loss: 0.5844
Epoch 3/5
464/464 ━━━━━━━━━━━━━━━━━━━━ 1455s 3s/step - loss: 0.5849 - val_loss: 0.5839
Epoch 4/5
464/464 ━━━━━━━━━━━━━━━━━━━━ 1288s 3s/step - loss: 0.5851 - val_loss: 0.5837
Epoch 5/5
464/464 ━━━━━━━━━━━━━━━━━━━━ 1262s 3s/step - loss: 0.5844 - val_loss: 0.5835


In [5]:
# Step 5: Predict on validation set
reconstructed_images = stcae_model.predict(val_images)

# Calculate reconstruction error
reconstruction_error = np.mean(np.square(val_images - reconstructed_images), axis=(1, 2, 3))

# Set a threshold for anomaly detection
threshold = np.percentile(reconstruction_error, 95)

# Predict anomalies
predicted_anomalies = reconstruction_error > threshold

# Evaluate accuracy
accuracy = np.mean(predicted_anomalies == val_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')


116/116 ━━━━━━━━━━━━━━━━━━━━ 96s 821ms/step
Accuracy: 67.19%
